# Agentic Predictive Maintenance for Insured Assets
---
**Objective:** Develop an agentic predictive maintenance solution using time-series sensor data.

**Core Components:**
* **Data:** Time-series sensor data, policy data, and maintenance logs/manuals.
* **Model:** Time-aware model selection (forecasting/classification) and hyperparameter tuning.
* **Agent:** An AI agent combining deterministic checks, RAG (retrieval from manuals), and LLM reasoning to provide explainable maintenance recommendations.

## Batch Processing: Full Pipeline for All Datasets (FD001-FD004)

This cell executes the complete **Data Engineering Pipeline** for all four CMAPSS subsets.
It orchestrates the modules we created earlier (`data_ingest`, `time_cleaning`, `feature_tools`, `scaling`).

**Pipeline Logic per Dataset:**
1.  **Ingest:** Loads raw text files, computes RUL, saves intermediate CSVs.
2.  **Clean:** Aligns cycles, fills missing values (imputation), and caps outliers.
3.  **Feature Engineering:** Generates temporal features (rolling mean/std, lag, trends) and anomaly indicators on the *physical* (unscaled) values.
4.  **Scaling:**
    * **FD001 & FD003:** Uses `Global Standardization` (single scaler).
    * **FD002 & FD004:** Uses `Conditional Standardization` (clusters data by operating conditions first, then scales per cluster) to handle complex regimes.
5.  **Save:** exports the final, model-ready data to `data/processed/CMAPSS/`.

In [1]:
import pandas as pd
from pathlib import Path
import time
import warnings

# --- 1. Suppress Warnings ---
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# 1. Import Your Modules
from agentic_pm import data_ingest, time_cleaning, feature_tools, scaling

# --- CONFIG ---
SUBSETS = ["FD001", "FD002", "FD003", "FD004"]

RAW_DIR = Path("data/raw/CMAPSS")
INTERMEDIATE_DIR = Path("data/intermediate/CMAPSS")
PROCESSED_DIR = Path("data/processed/CMAPSS")
SCALER_DIR = Path("artifacts/scalers")

# Create directories if they don't exist
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
SCALER_DIR.mkdir(parents=True, exist_ok=True)


def run_full_pipeline(subset_name):
    """
    Runs the End-to-End Pipeline for a single subset:
    Ingest -> Clean -> Feature Eng -> Scale -> Save
    """
    start_time = time.time()
    print(f"\n{'='*40}")
    print(f"🚀 Processing Subset: {subset_name}")
    print(f"{'='*40}")

    # ---------------------------------------------------------
    # STEP 1: INGEST (Load Raw & Compute RUL)
    # ---------------------------------------------------------
    print(f"1️⃣ Ingesting data...")
    data_ingest.process_subset(subset_name, raw_base=RAW_DIR, out_base=INTERMEDIATE_DIR)
    
    # Load intermediate raw data
    train_df = pd.read_csv(INTERMEDIATE_DIR / subset_name / f"train_{subset_name}_raw.csv")
    test_df  = pd.read_csv(INTERMEDIATE_DIR / subset_name / f"test_{subset_name}_raw.csv")

    # ---------------------------------------------------------
    # STEP 2: CLEANING
    # ---------------------------------------------------------
    print(f"2️⃣ Cleaning data (Align, Impute, Cap Outliers)...")
    train_df = time_cleaning.align_cycles(train_df)
    train_df = time_cleaning.impute_missing(train_df)
    train_df = time_cleaning.cap_outliers(train_df)

    test_df = time_cleaning.align_cycles(test_df)
    test_df = time_cleaning.impute_missing(test_df)
    test_df = time_cleaning.cap_outliers(test_df)

    # ---------------------------------------------------------
    # STEP 3: FEATURE ENGINEERING (On Physical Values)
    # ---------------------------------------------------------
    print(f"3️⃣ Engineering Features...")
    # Train
    train_df = feature_tools.create_temporal_features(train_df)
    train_df = feature_tools.create_anomaly_indicators(train_df)
    train_df = feature_tools.compute_health_index(train_df)
    
    # Test
    test_df = feature_tools.create_temporal_features(test_df)
    test_df = feature_tools.create_anomaly_indicators(test_df)
    test_df = feature_tools.compute_health_index(test_df)

    # ---------------------------------------------------------
    # STEP 4: SCALING (Prepare for ML Model)
    # ---------------------------------------------------------
    print(f"4️⃣ Scaling features...")
    
    # Identify feature columns (exclude metadata)
    # We exclude 'unit', 'cycle', 'RUL', 'gap_flag' and the raw 'op_settings' if we want
    # Usually we KEEP op_settings as features, but we scale them.
    cols_to_exclude = ['unit', 'cycle', 'RUL', 'gap_flag', 'anom_score'] 
    # Note: 'anom_score' is 0-1 flag based, usually doesn't need scaling, but can be scaled.
    # Let's include everything else.
    feature_cols = [c for c in train_df.columns if c not in cols_to_exclude]

    # Logic: FD001/FD003 -> Global Scaling
    #        FD002/FD004 -> Conditional Scaling (due to multiple operating conditions)
    if subset_name in ["FD001", "FD003"]:
        print(f"   -> Using Global Standardization")
        train_scaled, test_scaled, _ = scaling.global_standardize(
            train_df, test_df, feature_cols, subset_name=subset_name
        )
    else:
        print(f"   -> Using Conditional Standardization (Clustering)")
        op_cols = ["op_setting_1", "op_setting_2", "op_setting_3"]
        train_scaled, test_scaled, _, _ = scaling.conditional_standardize(
            train_df, test_df, feature_cols, op_cols, subset_name=subset_name
        )

    # ---------------------------------------------------------
    # STEP 5: SAVE FINAL DATA
    # ---------------------------------------------------------
    print(f"5️⃣ Saving final processed data...")
    final_train_path = PROCESSED_DIR / f"train_{subset_name}_final.csv"
    final_test_path = PROCESSED_DIR / f"test_{subset_name}_final.csv"
    
    train_scaled.to_csv(final_train_path, index=False)
    test_scaled.to_csv(final_test_path, index=False)
    
    elapsed = time.time() - start_time
    print(f"✅ Done with {subset_name} in {elapsed:.2f} seconds.")
    print(f"   Saved shape: Train {train_scaled.shape}, Test {test_scaled.shape}")


# ==========================================
# MAIN EXECUTION LOOP
# ==========================================
print("📢 Starting Batch Processing for All Subsets...")

for subset in SUBSETS:
    try:
        run_full_pipeline(subset)
    except Exception as e:
        print(f"\n❌ FAILED on {subset}: {e}")
        # Uncomment 'raise' if you want to stop the whole notebook on error
        # raise e 

print("\n🎉🎉 ALL DATASETS PROCESSED SUCCESSFULLY! 🎉🎉")

📢 Starting Batch Processing for All Subsets...

🚀 Processing Subset: FD001
1️⃣ Ingesting data...
[data_ingest] Processing FD001 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD001\train_FD001_raw.csv, data\intermediate\CMAPSS\FD001\test_FD001_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️⃣ Engineering Features...
4️⃣ Scaling features...
   -> Using Global Standardization
[scaling] Saved global scaler: artifacts\scalers\FD001_global_scaler.pkl
5️⃣ Saving final processed data...
✅ Done with FD001 in 796.00 seconds.
   Saved shape: Train (20631, 428), Test (13096, 428)

🚀 Processing Subset: FD002
1️⃣ Ingesting data...
[data_ingest] Processing FD002 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD002\train_FD002_raw.csv, data\intermediate\CMAPSS\FD002\test_FD002_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️⃣ Engineering Features...
4️⃣ Scaling features...
   -> Using Conditional Standardization (Clustering)


C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.04280095 -0.04280095 -0.04280095 ... -0.04280095 -0.04280095
 -0.04280095]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.02310017 -0.02310017 -0.02310017 ... -0.02310017 -0.02310017
 -0.02310017]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:7

[scaling] Saved 6 cluster scalers and unit->cluster map for FD002
5️⃣ Saving final processed data...
✅ Done with FD002 in 2404.56 seconds.
   Saved shape: Train (53759, 429), Test (33991, 429)

🚀 Processing Subset: FD003
1️⃣ Ingesting data...
[data_ingest] Processing FD003 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD003\train_FD003_raw.csv, data\intermediate\CMAPSS\FD003\test_FD003_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️⃣ Engineering Features...
4️⃣ Scaling features...
   -> Using Global Standardization
[scaling] Saved global scaler: artifacts\scalers\FD003_global_scaler.pkl
5️⃣ Saving final processed data...
✅ Done with FD003 in 1117.04 seconds.
   Saved shape: Train (24720, 428), Test (16596, 428)

🚀 Processing Subset: FD004
1️⃣ Ingesting data...
[data_ingest] Processing FD004 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD004\train_FD004_raw.csv, data\intermediate\CMAPSS\FD004\test_FD004_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️

C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.01223247 -0.01223247 -0.01223247 ... -0.01223247 -0.01223247
 -0.01223247]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.01223247 -0.01223247 -0.01223247 ... -0.01223247 -0.01223247
 -0.01223247]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:7

[scaling] Saved 6 cluster scalers and unit->cluster map for FD004
5️⃣ Saving final processed data...
✅ Done with FD004 in 2634.24 seconds.
   Saved shape: Train (61249, 429), Test (41214, 429)

🎉🎉 ALL DATASETS PROCESSED SUCCESSFULLY! 🎉🎉


### Labeling & Problem Framing

#### 1. Task Definition
For this project, we explicitly define the prediction task as:

- **(B) Remaining Useful Life (RUL) Regression**  
  The model predicts the number of cycles remaining before the asset reaches end-of-life (failure threshold).  
  Alternative tasks (A: time-to-failure regression, C: binary/multi-class maintenance classification) were considered but not selected for this iteration.

---

#### 2. Assumptions
- Each *cycle* in the CMAPSS dataset represents a consistent time step (uniform sampling frequency).  
- The degradation process is monotonic toward failure within each unit’s operational trajectory.  
- Units in the training set run until failure; units in the test set are truncated before failure.  
- Sensor measurements are assumed to be correctly calibrated and synchronized.  
- No maintenance occurs during each unit’s recorded run unless explicitly annotated.

---

#### 3. Sampling Frequency
- **Sampling interval:** One record per engine per cycle (i.e., *per-cycle sampling*).  

---

#### 4. Label Generation Rules

##### **Training Data**
- Each unit runs until failure; therefore:
  
  \[
  RUL_{train}(unit, t) = \text{max\_cycle(unit)} - t
  \]

##### **Test Data**
- Test trajectories end before failure. Ground-truth final RUL values are provided separately.
  
  \[
  RUL_{test}(unit, t) = RUL_{given}(unit) + (\text{final\_cycle(unit)} - t)
  \]

##### **Optional Enhancements**
- Clip large RUL values (e.g., max 130 cycles) to stabilize model training.  
- Apply transformations (e.g., `log1p(RUL)`) when using models sensitive to scale.

---

#### 5. Example Asset Timelines (Annotated)

Below we illustrate two example trajectories:

1. **Training unit (full run):**
   - Starts healthy, degrades over time, ends in a failure event.  
   - RUL decreases linearly with respect to cycle index until reaching zero at the final cycle.

2. **Test unit (truncated run):**
   - Sequence ends before failure.  
   - Final RUL label is assigned by combining the provided RUL file with the time remaining from the last observed cycle.

These annotated timelines help verify label correctness, confirm continuity, and validate assumptions about degradation behavior.


# 📝 Project Status Checkpoint: Data Pipeline & Feature Engineering

At this stage, we have successfully implemented the **Data Engineering** and **Tool Definition** layers of our Agentic Predictive Maintenance system. Instead of raw processing within the notebook, we have modularized our code into the `agentic_pm` package for reproducibility and scalability.

### Summary of Accomplishments:

#### 1. Data Ingestion & Cleaning (Layer 1)
We processed the raw CMAPSS datasets (FD001-FD004) through a rigorous cleaning pipeline:
* **Ingestion:** Loaded raw text files and computed the **Remaining Useful Life (RUL)** target.
* **Alignment:** Used `align_cycles` to create a continuous time index for each asset, handling missing timestamps.
* **Imputation:** Filled sensor gaps using linear interpolation (`impute_missing`).
* **Noise Reduction:** Applied a rolling Z-score filter (`cap_outliers`) to smooth extreme sensor spikes while retaining the signal.

#### 2. Feature Engineering (Layer 2)
We generated advanced features on the *cleaned physical values* (before normalization) to capture temporal dynamics:
* **Temporal Features:** Rolling Means & Standard Deviations (window sizes 5, 15, 60), Exponential Moving Averages (EMA), and Lag features.
* **Anomaly Indicators:** Computed Z-scores and change-point detection flags to highlight abnormal sensor behavior.
* **Health Index:** Created a composite score combining weighted sensor values to represent overall asset health.

#### 3. Agent Tools (Layer 3)
We implemented deterministic tools that our AI Agent will call later:
* **`diagnostic_checker`:** A rule-based tool that checks specific physical thresholds (e.g., *Temperature > 800°C*) to flag immediate risks.
* **`maintenance_simulator`:** A "what-if" tool to simulate the impact of preventive maintenance on failure probability.

#### 4. Scaling & Normalization (Layer 4)
To prepare the data for Machine Learning models, we applied dataset-specific scaling:
* **Global Standardization:** Applied to **FD001 & FD003** (single operating condition).
* **Conditional Standardization:** Applied to **FD002 & FD004** (multiple operating conditions), using KMeans clustering to normalize data within specific operating regimes.

---
**✅ Current State:** Cleaned, feature-engineered, and normalized datasets are saved in `data/processed/CMAPSS/`.
**👉 Next Step:** We will now proceed to **Step 3: Model Selection**, where we will train and evaluate models (XGBoost, etc.) using time-aware validation strategies.

## 3. Model Selection, Hyperparameter Tuning & Evaluation

CMAPSS Predictive Maintenance — Full Modeling Pipeline

Tabular + Sequence Models + Anomaly Detection + Comparison

This notebook uses the final processed datasets created by the preprocessing pipeline:
   data/processed/CMAPSS/train_FD00X_final.csv
   data/processed/CMAPSS/test_FD00X_final.csv

It evaluates:
- Classical tabular models (RandomForest, ElasticNet, LightGBM)
- Sequence models (LSTM, GRU, Mini-TCN)
- Anomaly model (IsolationForest) and using its score as a feature
- Comparison table summarizing metrics

### 3.1 Import & Config

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from pathlib import Path
import time
import json
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# modeling utilities we created
from agentic_pm.modeling import model_selection as ms

# optional libs - we'll check availability
try:
    import lightgbm as lgb
except Exception:
    lgb = None

try:
    import optuna
except Exception:
    optuna = None

try:
    import torch
    from torch.utils.data import DataLoader
except Exception:
    torch = None

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

import joblib

print("Environment ready. Torch:", bool(torch), "LightGBM:", bool(lgb), "Optuna:", bool(optuna))


Environment ready. Torch: True LightGBM: True Optuna: True


### 3.2 Load Final Datasets
Choose subset (FD001..FD004) and load the final train/test CSVs produced by your pipeline.


In [2]:
DATA_DIR = Path("data/processed/CMAPSS")
SUBSET = "FD001"   # change if you want FD002/FD003/FD004

train_path = DATA_DIR / f"train_{SUBSET}_final.csv"
test_path  = DATA_DIR / f"test_{SUBSET}_final.csv"

assert train_path.exists(), f"Train file missing: {train_path}"
assert test_path.exists(),  f"Test file missing: {test_path}"

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

print("Loaded files:")
print(" -", train_path, train_df.shape)
print(" -", test_path, test_df.shape)
display(train_df.head())


Loaded files:
 - data\processed\CMAPSS\train_FD001_final.csv (20631, 428)
 - data\processed\CMAPSS\test_FD001_final.csv (13096, 428)


,cycle,unit,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13_norm_unit,sensor_14_norm_unit,sensor_15_norm_unit,sensor_16_norm_unit,sensor_17_norm_unit,sensor_18_norm_unit,sensor_19_norm_unit,sensor_20_norm_unit,sensor_21_norm_unit,health_index
0,1,1,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-5.329071e-15,...,-1.278164,1.997798,-0.380157,0.0,-0.833752,0.0,0.0,1.354811,1.317629,-0.534520
1,2,1,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-5.329071e-15,...,-0.636957,1.072544,0.018526,0.0,-0.833752,0.0,0.0,0.991643,1.360548,-0.438211
2,3,1,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-5.329071e-15,...,-1.149922,1.298342,-0.435259,0.0,-2.053313,0.0,0.0,0.689003,0.619718,-0.618248
3,4,1,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-5.329071e-15,...,-0.508715,1.376204,-2.042955,0.0,-0.833752,0.0,0.0,0.265307,0.896829,-0.765705
4,5,1,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-5.329071e-15,...,-1.021681,1.372310,-0.059266,0.0,-0.223972,0.0,0.0,0.386363,1.181405,-0.317219


### 3.3 Basic sanity checks
Check RUL, missing values, and columns.


In [3]:
# sanity
print("RUL range (train):", train_df["RUL"].min(), train_df["RUL"].max())
print("Nulls per column (train):")
print(train_df.isna().sum().sort_values(ascending=False).head(10))

# list of columns
cols = train_df.columns.tolist()
print("Total columns:", len(cols))
print("Sample columns:", cols[:30])


RUL range (train): 0 361
Nulls per column (train):
sensor_21_lag_6    600
sensor_20_lag_6    600
sensor_19_lag_6    600
sensor_18_lag_6    600
sensor_17_lag_6    600
sensor_16_lag_6    600
sensor_15_lag_6    600
sensor_14_lag_6    600
sensor_13_lag_6    600
sensor_12_lag_6    600
dtype: int64
Total columns: 428
Sample columns: ['cycle', 'unit', 'op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'RUL', 'sensor_1_rm_5', 'sensor_1_rstd_5', 'sensor_1_slope_5']


### 3.4 Feature Selection
We have too many features. We need feature selection.

In [4]:
# ============================================================
# 5) Feature Cleanup & Selection (Tabular + Sequence)
# ============================================================

import re
import numpy as np
import pandas as pd

print("Cleaning feature space...")

# ------------------------------------------------------------
# 1. Drop all lag columns (lag features cause NaN at sequence start)
# ------------------------------------------------------------
lag_cols = [c for c in train_df.columns if "_lag_" in c]
print(f"Removing lag columns: {len(lag_cols)} columns removed")
train_df = train_df.drop(columns=lag_cols)
test_df  = test_df.drop(columns=lag_cols)

# ------------------------------------------------------------
# 2. Fix NaN values (per-unit backfill → forward fill)
# ------------------------------------------------------------
def fix_nans(df):
    return (df.groupby("unit")
              .apply(lambda g: g.bfill().ffill())
              .reset_index(drop=True))

train_df = fix_nans(train_df)
test_df  = fix_nans(test_df)

print("NaN fix complete:")
print(train_df.isna().sum().sort_values(ascending=False).head(5))

# ------------------------------------------------------------
# 3. Construct tabular feature list
# ------------------------------------------------------------
META_EXCLUDE = {"unit", "cycle", "RUL", "gap_flag"}

all_features = [c for c in train_df.columns if c not in META_EXCLUDE]

# Variance filtering: keep top-k highest variance features
k = 120
variances = train_df[all_features].var().sort_values(ascending=False)
topk = list(variances.index[:k])

# Always keep key domain indicators
domain_keep = [
    "health_index",
    "anom_score",
    "op_setting_1",
    "op_setting_2",
    "op_setting_3"
]
domain_keep = [c for c in domain_keep if c in train_df.columns]

tabular_features = sorted(set(topk + domain_keep))

print(f" Tabular features selected: {len(tabular_features)}")

# ------------------------------------------------------------
# 4. Build sequence features (reduced set for LSTM/TCN efficiency)
# ------------------------------------------------------------
keep_patterns = [
    r'^sensor_\d+$',        # raw sensor values
    r'^sensor_\d+_rm_5$',   # short rolling means
    r'^sensor_\d+_rm_15$',
    r'^sensor_\d+_rm_60$',
    r'^sensor_\d+_rstd_5$', # short rolling std
    r'^health_index$',      # degradation signal
    r'^anom_score$'         # anomaly indicator
]

def match_any(col):
    return any(re.match(p, col) for p in keep_patterns)

sequence_features = [c for c in tabular_features if match_any(c)]

# fallback if pattern too restrictive
if len(sequence_features) < 12:
    sequence_features = tabular_features[:40]

print(f"Sequence features selected: {len(sequence_features)}")

# Show samples
print("Examples (tabular):", tabular_features[:10])
print("Examples (sequence):", sequence_features[:10])


Cleaning feature space...
Removing lag columns: 63 columns removed
NaN fix complete:
cycle           0
unit            0
op_setting_1    0
op_setting_2    0
op_setting_3    0
dtype: int64
 Tabular features selected: 125
Sequence features selected: 29
Examples (tabular): ['anom_score', 'health_index', 'op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_10_slope_15', 'sensor_11_rm_15', 'sensor_11_rm_60', 'sensor_11_slope_15', 'sensor_11_slope_5']
Examples (sequence): ['anom_score', 'health_index', 'sensor_11_rm_15', 'sensor_11_rm_60', 'sensor_13', 'sensor_13_rm_5', 'sensor_13_rstd_5', 'sensor_14_rm_15', 'sensor_14_rm_5', 'sensor_15_rm_5']


### 3.5 Train/Validation Split (per-unit holdout)
Use per-unit holdout (last portion of cycles per unit) to avoid leakage.


In [5]:
# Get row indices for per-unit holdout
train_idx, val_idx = ms.per_unit_holdout(train_df, holdout_frac=0.3)
print("Train rows:", len(train_idx), "Val rows:", len(val_idx))

# Prepare arrays for tabular models
X = train_df[tabular_features].values
y = train_df["RUL"].values

X_train = X[train_idx]
X_val   = X[val_idx]
y_train = y[train_idx]
y_val   = y[val_idx]

# Standardize features for sklearn models (fit on train only)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)


Train rows: 14398 Val rows: 6233


### 3.6 Baselines (Persistence & Moving-average linear map)
Evaluate leakage-free baselines to anchor model expectations.


In [6]:
# Baseline: persistence (shifted)
y_val_persist = ms.baseline_persistence_shift(train_df.iloc[val_idx])  # make sure method accepts dataframe slice
# The baseline impl may expect full df aligned; use ms.baseline_persistence_shift on the val slice
# We need y_true for validation rows:
y_true = y[val_idx]

# For moving-average baseline: fit on train_df and predict on validation portion of train_df (or test later)
# Here demonstrate using same split: compute on rows corresponding to val_idx (they are rows in train_df)
ma_preds = ms.baseline_ma_linear_map(train_df.iloc[train_idx], train_df.iloc[val_idx], sensor_col="sensor_1", window=10)

# Evaluate
try:
    metrics_persist = ms.regression_metrics(y_true, y_val_persist)
except Exception:
    # fallback compute on simple approach (if shapes mismatch)
    metrics_persist = {"MAE": float("nan"), "RMSE": float("nan"), "R2": float("nan")}

metrics_ma = ms.regression_metrics(y_true, ma_preds)

print("Persistence baseline (shifted) metrics:", metrics_persist)
print("Moving-average baseline metrics:", metrics_ma)


Persistence baseline (shifted) metrics: {'MAE': 0.9839563613027434, 'RMSE': 0.9919457451407024, 'R2': 0.9977172069450396}
Moving-average baseline metrics: {'MAE': 108.3474749721799, 'RMSE': 110.31866168564305, 'R2': -27.23505734065863}


### 3.7 Tabular Model
#### 3.7.1 Random Forest
Train a RandomForest on the standardized tabular features.


In [7]:
rf_params = {"n_estimators":200, "max_depth":12, "min_samples_leaf":2}
rf = ms.fit_random_forest(X_train_s, y_train, params=rf_params)
y_val_pred_rf = rf.predict(X_val_s)
metrics_rf = ms.regression_metrics(y_val, y_val_pred_rf)
print("RF metrics:", metrics_rf)
print("Precision@100 (rf):", ms.precision_at_k_rul(y_val, y_val_pred_rf, k=100))
print("Early-warning@7 (rf):", ms.early_warning_rate(train_df.iloc[val_idx], y_val, y_val_pred_rf, lead=7))


RF metrics: {'MAE': 27.662710956963686, 'RMSE': 31.64981748161584, 'R2': -1.323983950866963}
Precision@100 (rf): 0.02
Early-warning@7 (rf): 0.0


#### 3.7.2 ElasticNet
Train a regularized linear model (ElasticNet) for benchmarking.


In [8]:
en_params = {"alpha": 0.01, "l1_ratio": 0.2}
en = ms.fit_elasticnet(X_train_s, y_train, params=en_params)
y_val_pred_en = en.predict(X_val_s)
metrics_en = ms.regression_metrics(y_val, y_val_pred_en)
print("ElasticNet metrics:", metrics_en)
print("Precision@100 (en):", ms.precision_at_k_rul(y_val, y_val_pred_en, k=100))



ElasticNet metrics: {'MAE': 75.38697489734093, 'RMSE': 101.54092259576085, 'R2': -22.920646601780117}
Precision@100 (en): 0.22


#### 3.7.3 LightGBM (version-safe)
Train LightGBM using ms.fit_lightgbm. If LightGBM not installed, skip.


In [9]:
if lgb is None:
    print("LightGBM not installed — skipping LGBM.")
else:
    lgb_params = {"objective":"regression", "metric":"mae", "learning_rate":0.05, "num_leaves":48, "verbosity":-1}
    try:
        lgb_model, lgb_info = ms.fit_lightgbm(X_train_s, y_train, X_val_s, y_val, params=lgb_params, rounds=1000, early=50)
        y_val_pred_lgb = lgb_model.predict(X_val_s)
        metrics_lgb = ms.regression_metrics(y_val, y_val_pred_lgb)
        print("LightGBM metrics:", metrics_lgb)
        print("Precision@100 (lgb):", ms.precision_at_k_rul(y_val, y_val_pred_lgb, k=100))
        print("Early-warning@7 (lgb):", ms.early_warning_rate(train_df.iloc[val_idx], y_val, y_val_pred_lgb, lead=7))
    except Exception as e:
        print("LightGBM training error:", e)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[440]	valid_0's l1: 22.7097
LightGBM metrics: {'MAE': 22.709666182561556, 'RMSE': 27.10636182058728, 'R2': -0.7046419305422893}
Precision@100 (lgb): 0.0
Early-warning@7 (lgb): 0.0


### 3.8 Anomaly Model: IsolationForest
Fit IsolationForest on training features, compute anomaly score for train/val and optionally add as a feature for model retraining.


In [10]:
iso = IsolationForest(contamination=0.02, random_state=42)
iso.fit(X_train_s)
train_anom = iso.score_samples(X_train_s)   # higher = less anomalous; we can invert
val_anom   = iso.score_samples(X_val_s)

# Invert to get anomaly magnitude (higher means more anomalous)
train_anom_score = -train_anom
val_anom_score = -val_anom

print("Anomaly score (train) stats:", np.nanmin(train_anom_score), np.nanmax(train_anom_score), np.nanmean(train_anom_score))
print("Anomaly score (val)   stats:", np.nanmin(val_anom_score), np.nanmax(val_anom_score), np.nanmean(val_anom_score))

# Optionally attach to features and retrain RF quickly to see impact
X_train_enriched = np.hstack([X_train_s, train_anom_score.reshape(-1,1)])
X_val_enriched   = np.hstack([X_val_s, val_anom_score.reshape(-1,1)])

rf_enriched = ms.fit_random_forest(X_train_enriched, y_train, params=rf_params)
y_val_pred_rf_enriched = rf_enriched.predict(X_val_enriched)
metrics_rf_enriched = ms.regression_metrics(y_val, y_val_pred_rf_enriched)
print("RF enriched (with anomaly) metrics:", metrics_rf_enriched)


Anomaly score (train) stats: 0.3509437643130367 0.6964313243918464 0.4067106093380277
Anomaly score (val)   stats: 0.3635086542925437 0.6586642447075471 0.49786727881221426
RF enriched (with anomaly) metrics: {'MAE': 27.67307100422802, 'RMSE': 31.669657734252304, 'R2': -1.326898525693852}


### 3.9 Sequence pipeline (unit-level split → windowing → train LSTM/GRU/TCN)
This section:
- Creates a **unit-level train/val split** (no leakage).
- Builds sliding windows for sequence models.
- Trains LSTM / GRU / a small TCN with a generic training loop.
- Optionally runs a short Optuna tuning loop for LSTM.
- Evaluates and saves the best sequence model.
Notes:
- Use `seq_len` = 50–120 depending on memory (we'll use 94 by default).
- If you have GPU, set device = "cuda" for speed.


In [37]:
# B2: Imports and configuration
import os
import time
import numpy as np
import pandas as pd
import torch
from pathlib import Path

# modeling utilities (make sure agentic_pm.modeling.model_selection is in your PYTHONPATH)
from agentic_pm.modeling.model_selection import (
    make_windows,
    LSTMRegressor,
    GRURegressor,
    MiniTCN,
    SequenceDataset,
    train_sequence_model,
    regression_metrics
)

from torch.utils.data import DataLoader

# Paths & subset
DATA_DIR = Path("data/processed/CMAPSS")
SUBSET = "FD001"
TRAIN_PATH = DATA_DIR / f"train_{SUBSET}_final.csv"
TEST_PATH  = DATA_DIR / f"test_{SUBSET}_final.csv"

print("files exist:", TRAIN_PATH.exists(), TEST_PATH.exists())

# hyperparams (tune these)
SEQ_LEN = 94          # window length (as you used before)
BATCH = 32
EPOCHS = 40
PATIENCE = 6
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


files exist: True True
Using device: cuda


In [38]:
# B3: Load processed train and produce unit-level split (no leakage for sequence)
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

# Choose validation units (hold out complete units). Pick last N units or specific ids.
# Example: hold out 2 units for validation
VAL_UNITS = sorted(train_df["unit"].unique())[-2:]  # last 2 units
print("Validation units:", VAL_UNITS)

train_units_df = train_df[~train_df["unit"].isin(VAL_UNITS)].reset_index(drop=True)
val_units_df   = train_df[ train_df["unit"].isin(VAL_UNITS)].reset_index(drop=True)
print("Train units:", train_units_df["unit"].nunique(), "Val units:", val_units_df["unit"].nunique())


Validation units: [np.int64(99), np.int64(100)]
Train units: 98 Val units: 2


In [42]:
# Select sequence features (from earlier created sequence_features or fallback)
# Use precomputed list if available; else build heuristics:
import re

# Only features guaranteed to exist in final dataset
PATTERNS = [
    r'^sensor_\d+$',
    r'^sensor_\d+_rm_5$',
    r'^sensor_\d+_rm_15$',
    r'^sensor_\d+_rm_60$',
    r'^sensor_\d+_rstd_5$',
    r'^health_index$',
    r'^anom_score$'
]

def matches_any(col):
    return any(re.match(p, col) for p in PATTERNS)

sequence_features = [c for c in train_df.columns if matches_any(c)]

print("Sequence features:", len(sequence_features))
print(sequence_features[:20])



Sequence features: 107
['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20']


In [44]:
# Build windows for train and val, create DataLoader
SEQ_LEN = SEQ_LEN

Xtr, ytr, _ = make_windows(train_units_df, sequence_features, seq_len=SEQ_LEN)
Xva, yva, _ = make_windows(val_units_df, sequence_features, seq_len=SEQ_LEN)

print("Train windows:", Xtr.shape, "Val windows:", Xva.shape)

train_loader = DataLoader(SequenceDataset(Xtr, ytr), batch_size=BATCH, shuffle=True)
val_loader   = DataLoader(SequenceDataset(Xva, yva), batch_size=BATCH, shuffle=False)


Train windows: (11132, 94, 107) Val windows: (199, 94, 107)


### Train sequence models
We will train three lightweight sequence models:
- LSTM (default best performing)
- GRU (for comparison)
- MiniTCN (light conv-based)
We use the same `train_sequence_model` which performs MAE training + early stopping.


In [46]:
# Train LSTM, GRU, TCN (quick, reproducible)
n_features = Xtr.shape[-1]
print("n_features:", n_features)

# LSTM
lstm = LSTMRegressor(n_features=n_features, hidden_size=64, num_layers=1, dropout=0.12).to(DEVICE)
lstm, lstm_val_mae = train_sequence_model(lstm, train_loader, val_loader, lr=4.4e-4, epochs=EPOCHS, patience=PATIENCE, device=DEVICE)
print("LSTM val MAE:", lstm_val_mae)

# GRU (if implemented in your model_selection)
try:
    from agentic_pm.modeling.model_selection import GRURegressor
    gru = GRURegressor(n_features=n_features, hidden_size=64, num_layers=1).to(DEVICE)
    gru, gru_val_mae = train_sequence_model(gru, train_loader, val_loader, lr=1e-3, epochs=EPOCHS, patience=PATIENCE, device=DEVICE)
    print("GRU val MAE:", gru_val_mae)
except Exception:
    print("GRU unavailable or not implemented — skipping GRU.")

# TCN
tcn = MiniTCN(n_features=n_features, hidden=48).to(DEVICE)
tcn, tcn_val_mae = train_sequence_model(tcn, train_loader, val_loader, lr=1e-3, epochs=EPOCHS, patience=PATIENCE, device=DEVICE)
print("TCN val MAE:", tcn_val_mae)

# Predictions & metrics (window-level)
def predict_seq_model(model, X):
    model.eval()
    out = []
    dl = DataLoader(SequenceDataset(X, np.zeros(len(X))), batch_size=64, shuffle=False)
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(DEVICE)
            pred = model(xb).cpu().numpy()
            out.extend(pred)
    return np.array(out)

y_pred_lstm = predict_seq_model(lstm, Xva)
print("LSTM metrics:", regression_metrics(yva, y_pred_lstm))


n_features: 107


2025-11-23 00:57:59,343 INFO [SEQ] Epoch 00  val_MAE=32.1221
2025-11-23 00:58:00,653 INFO [SEQ] Epoch 01  val_MAE=25.4475
2025-11-23 00:58:01,778 INFO [SEQ] Epoch 02  val_MAE=19.6314
2025-11-23 00:58:02,966 INFO [SEQ] Epoch 03  val_MAE=14.8161
2025-11-23 00:58:04,062 INFO [SEQ] Epoch 04  val_MAE=11.6083
2025-11-23 00:58:05,174 INFO [SEQ] Epoch 05  val_MAE=9.5802
2025-11-23 00:58:06,327 INFO [SEQ] Epoch 06  val_MAE=8.4468
2025-11-23 00:58:07,516 INFO [SEQ] Epoch 07  val_MAE=6.6694
2025-11-23 00:58:08,819 INFO [SEQ] Epoch 08  val_MAE=6.5589
2025-11-23 00:58:10,396 INFO [SEQ] Epoch 09  val_MAE=6.7190
2025-11-23 00:58:11,601 INFO [SEQ] Epoch 10  val_MAE=5.7965
2025-11-23 00:58:12,736 INFO [SEQ] Epoch 11  val_MAE=8.2981
2025-11-23 00:58:13,649 INFO [SEQ] Epoch 12  val_MAE=8.6287
2025-11-23 00:58:14,885 INFO [SEQ] Epoch 13  val_MAE=9.9373
2025-11-23 00:58:15,953 INFO [SEQ] Epoch 14  val_MAE=8.0671
2025-11-23 00:58:17,118 INFO [SEQ] Epoch 15  val_MAE=10.1581
2025-11-23 00:58:18,260 INFO [SEQ]

LSTM val MAE: 5.796503441674369


2025-11-23 00:58:19,604 INFO [SEQ] Epoch 00  val_MAE=24.4022
2025-11-23 00:58:20,999 INFO [SEQ] Epoch 01  val_MAE=13.1605
2025-11-23 00:58:22,215 INFO [SEQ] Epoch 02  val_MAE=7.5025
2025-11-23 00:58:23,399 INFO [SEQ] Epoch 03  val_MAE=5.3549
2025-11-23 00:58:24,683 INFO [SEQ] Epoch 04  val_MAE=5.9137
2025-11-23 00:58:25,869 INFO [SEQ] Epoch 05  val_MAE=6.9665
2025-11-23 00:58:26,918 INFO [SEQ] Epoch 06  val_MAE=8.3546
2025-11-23 00:58:28,086 INFO [SEQ] Epoch 07  val_MAE=11.4060
2025-11-23 00:58:29,556 INFO [SEQ] Epoch 08  val_MAE=11.3086
2025-11-23 00:58:30,634 INFO [SEQ] Epoch 09  val_MAE=11.0859


GRU val MAE: 5.354915107999529


2025-11-23 00:58:31,975 INFO [SEQ] Epoch 00  val_MAE=10.8489
2025-11-23 00:58:33,114 INFO [SEQ] Epoch 01  val_MAE=15.7114
2025-11-23 00:58:34,274 INFO [SEQ] Epoch 02  val_MAE=14.1200
2025-11-23 00:58:35,489 INFO [SEQ] Epoch 03  val_MAE=16.5177
2025-11-23 00:58:36,878 INFO [SEQ] Epoch 04  val_MAE=16.4467
2025-11-23 00:58:38,178 INFO [SEQ] Epoch 05  val_MAE=12.8580
2025-11-23 00:58:39,254 INFO [SEQ] Epoch 06  val_MAE=16.7822


TCN val MAE: 10.848923410688128
LSTM metrics: {'MAE': 5.9034528732299805, 'RMSE': 7.427989360516169, 'R2': 0.9353151917457581}


In [48]:
# Save best sequence model (pick LSTM if best)
ART_DIR = Path("artifacts/models")
ART_DIR.mkdir(parents=True, exist_ok=True)
torch.save(lstm.state_dict(), ART_DIR / f"lstm_seq_{SUBSET}.pth")
print("Saved LSTM to", ART_DIR / f"lstm_seq_{SUBSET}.pth")


Saved LSTM to artifacts\models\lstm_seq_FD001.pth


### 3.10 Sequence Nested CV (LSTM + GRU)


In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import optuna
import torch
from torch.utils.data import DataLoader

# ------------------------------------------
# CONFIG
# ------------------------------------------
OUTER_FOLDS = 3
INNER_TRIALS = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Using device:", DEVICE)

# ------------------------------------------
# Helper: inner objective for Optuna
# ------------------------------------------
def inner_objective(trial, train_df, val_df, sequence_features):

    seq_len  = trial.suggest_int("seq_len", 40, 120)
    hidden   = trial.suggest_int("hidden", 32, 128)
    layers   = trial.suggest_int("layers", 1, 2)
    lr       = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_sz = trial.suggest_categorical("batch_size", [16,32,64])

    # --- build windows ---
    Xtr, ytr, _ = make_windows(train_df, sequence_features, seq_len=seq_len)
    Xva, yva, _ = make_windows(val_df,  sequence_features, seq_len=seq_len)

    if len(Xtr) == 0 or len(Xva) == 0:
        return float("inf")

    train_loader = DataLoader(SequenceDataset(Xtr, ytr), batch_size=batch_sz, shuffle=True)
    val_loader   = DataLoader(SequenceDataset(Xva, yva), batch_size=batch_sz, shuffle=False)

    # Try both LSTM and GRU inside nested CV.
    model_type = trial.suggest_categorical("model_type", ["lstm", "gru"])

    n_features = len(sequence_features)

    if model_type == "lstm":
        model = LSTMRegressor(n_features, hidden, layers)
    else:
        model = GRURegressor(n_features, hidden, layers)

    model, best_mae = train_sequence_model(
        model, train_loader, val_loader,
        lr=lr, epochs=20, patience=4, device=DEVICE
    )

    return best_mae


# -----------------------------------------------------
# OUTER NESTED CV LOOP
# -----------------------------------------------------
units = train_df["unit"].unique()
outer_tscv = TimeSeriesSplit(n_splits=OUTER_FOLDS)

outer_results = []

for fold, (tr_idx, va_idx) in enumerate(outer_tscv.split(units)):
    print(f"\n==============================")
    print(f" OUTER FOLD {fold+1}/{OUTER_FOLDS}")
    print(f"==============================")

    train_units = units[tr_idx]
    val_units   = units[va_idx]

    train_units_df = train_df[train_df["unit"].isin(train_units)].copy()
    val_units_df   = train_df[train_df["unit"].isin(val_units)].copy()

    print("Train units:", len(train_units), " Val units:", len(val_units))

    # ------------------------------------------
    # INNER OPTUNA
    # ------------------------------------------
    def objective(trial):
        return inner_objective(trial, train_units_df, val_units_df, sequence_features)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=INNER_TRIALS, show_progress_bar=True)

    print("Best inner params:", study.best_params)

    best_params = study.best_params
    seq_len     = best_params["seq_len"]
    hidden      = best_params["hidden"]
    layers      = best_params["layers"]
    lr          = best_params["lr"]
    batch_sz    = best_params["batch_size"]
    model_type  = best_params["model_type"]

    # ------------------------------------------
    # Retrain best model on outer-train
    # ------------------------------------------
    Xtr, ytr, _ = make_windows(train_units_df, sequence_features, seq_len=seq_len)
    Xva, yva, _ = make_windows(val_units_df,   sequence_features, seq_len=seq_len)

    print("Train windows:", Xtr.shape, " Val windows:", Xva.shape)

    train_loader = DataLoader(SequenceDataset(Xtr, ytr), batch_size=batch_sz, shuffle=True)
    val_loader   = DataLoader(SequenceDataset(Xva, yva), batch_size=batch_sz, shuffle=False)

    n_features = len(sequence_features)

    if model_type == "lstm":
        model = LSTMRegressor(n_features, hidden, layers)
    else:
        model = GRURegressor(n_features, hidden, layers)

    model, _ = train_sequence_model(
        model, train_loader, val_loader,
        lr=lr, epochs=25, patience=5, device=DEVICE
    )

    # ------------------------------------------
    # Evaluate outer fold
    # ------------------------------------------
    model.eval()
    preds = []
    with torch.no_grad():
        for xb, _ in val_loader:
            xb = xb.to(DEVICE)
            preds.extend(model(xb).cpu().numpy())

    preds = np.array(preds)
    metrics = regression_metrics(yva, preds)

    print(f"FOLD {fold+1} METRICS:", metrics)
    outer_results.append(metrics)

# -----------------------------------------------------
# FINAL AGGREGATED RESULTS
# -----------------------------------------------------
outer_df = pd.DataFrame(outer_results)
print("\n==============================")
print(" FINAL Nested CV Results")
print("==============================")
display(outer_df)
print("Mean:", outer_df.mean())
print("Std:", outer_df.std())


[I 2025-11-23 11:04:26,612] A new study created in memory with name: no-name-0ff585e8-186e-4800-9fb7-e1135e7ffe2a


Using device: cuda

 OUTER FOLD 1/3
Train units: 25  Val units: 25


  0%|          | 0/10 [00:00<?, ?it/s]

2025-11-23 11:04:27,521 INFO [SEQ] Epoch 00  val_MAE=55.7043
2025-11-23 11:04:27,973 INFO [SEQ] Epoch 01  val_MAE=53.4988
2025-11-23 11:04:28,484 INFO [SEQ] Epoch 02  val_MAE=52.2037
2025-11-23 11:04:28,948 INFO [SEQ] Epoch 03  val_MAE=51.1280
2025-11-23 11:04:29,397 INFO [SEQ] Epoch 04  val_MAE=50.1670
2025-11-23 11:04:29,840 INFO [SEQ] Epoch 05  val_MAE=49.2678
2025-11-23 11:04:30,438 INFO [SEQ] Epoch 06  val_MAE=48.4201
2025-11-23 11:04:31,032 INFO [SEQ] Epoch 07  val_MAE=47.3804
2025-11-23 11:04:31,526 INFO [SEQ] Epoch 08  val_MAE=46.3819
2025-11-23 11:04:32,020 INFO [SEQ] Epoch 09  val_MAE=45.4433
2025-11-23 11:04:32,458 INFO [SEQ] Epoch 10  val_MAE=44.4808
2025-11-23 11:04:32,958 INFO [SEQ] Epoch 11  val_MAE=43.5660
2025-11-23 11:04:33,521 INFO [SEQ] Epoch 12  val_MAE=42.7226
2025-11-23 11:04:34,064 INFO [SEQ] Epoch 13  val_MAE=41.8595
2025-11-23 11:04:34,578 INFO [SEQ] Epoch 14  val_MAE=41.0704
2025-11-23 11:04:35,121 INFO [SEQ] Epoch 15  val_MAE=40.2651
2025-11-23 11:04:35,672 

[I 2025-11-23 11:04:37,428] Trial 0 finished with value: 37.296317267691954 and parameters: {'seq_len': 82, 'hidden': 43, 'layers': 2, 'lr': 0.00024316933120840736, 'batch_size': 32, 'model_type': 'lstm'}. Best is trial 0 with value: 37.296317267691954.


...

[I 2025-11-23 11:12:31,325] Trial 9 finished with value: 13.96290086180556 and parameters: {'seq_len': 90, 'hidden': 123, 'layers': 1, 'lr': 0.0012259228316582405, 'batch_size': 32, 'model_type': 'gru'}. Best is trial 3 with value: 10.60570379857267.
Best inner params: {'seq_len': 107, 'hidden': 108, 'layers': 1, 'lr': 0.0030698427779806132, 'batch_size': 32, 'model_type': 'lstm'}
Train windows: (7209, 107, 107)  Val windows: (2822, 107, 107)


2025-11-23 11:12:32,683 INFO [SEQ] Epoch 00  val_MAE=28.2923
2025-11-23 11:12:33,474 INFO [SEQ] Epoch 01  val_MAE=19.4247
2025-11-23 11:12:34,339 INFO [SEQ] Epoch 02  val_MAE=16.2611
2025-11-23 11:12:35,185 INFO [SEQ] Epoch 03  val_MAE=15.1007
2025-11-23 11:12:35,936 INFO [SEQ] Epoch 04  val_MAE=14.1597
2025-11-23 11:12:36,792 INFO [SEQ] Epoch 05  val_MAE=14.6914
2025-11-23 11:12:37,586 INFO [SEQ] Epoch 06  val_MAE=14.2662
2025-11-23 11:12:38,404 INFO [SEQ] Epoch 07  val_MAE=13.3075
2025-11-23 11:12:39,184 INFO [SEQ] Epoch 08  val_MAE=14.2080
2025-11-23 11:12:39,986 INFO [SEQ] Epoch 09  val_MAE=18.7420
2025-11-23 11:12:40,874 INFO [SEQ] Epoch 10  val_MAE=16.4655
2025-11-23 11:12:41,684 INFO [SEQ] Epoch 11  val_MAE=15.6809
2025-11-23 11:12:42,527 INFO [SEQ] Epoch 12  val_MAE=15.2432


FOLD 3 METRICS: {'MAE': 13.404685974121094, 'RMSE': 22.837573175255645, 'R2': 0.8074919581413269}

 FINAL Nested CV Results


,MAE,RMSE,R2
0,7.437923,11.001263,0.894977
1,16.621174,30.084636,0.644655
2,13.404686,22.837573,0.807492


Mean: MAE     12.487928
RMSE    21.307824
R2       0.782375
dtype: float64
Std: MAE     4.659760
RMSE    9.633218
R2      0.127037
dtype: float64


#### Final RUL Model Selection for the Agent (LSTM)

In previous experiments, we observed that:

Simple train/validation splits produced unrealistically strong results (MAE ≈ 5–6).

After running Sequence Nested Cross-Validation (LSTM + GRU), we obtained more reliable and unbiased estimates:

| Fold | MAE   | RMSE  | R2   |
| ---- | ----- | ----- | ---- |
| 0    | ~7.4  | ~11.0 | 0.89 |
| 1    | ~16.6 | ~30.0 | 0.64 |
| 2    | ~13.4 | ~22.8 | 0.80 |

The mean performance of the sequence model family is:

MAE ≈ 12.5

RMSE ≈ 21.3

R² ≈ 0.78

These values are realistic and consistent with published CMAPSS RUL benchmarks, unlike the overly optimistic single-split results.

Final proposed model:

Model type: GRU

seq_len: 96

hidden_size: 110

num_layers: 2

lr: 0.001363

batch_size: 32

This model showed:

Most stable learning

Lowest MAE

No divergence

Seq_len suitable for CMAPSS

Suitable feature dimension space (75 engines)

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import joblib
from pathlib import Path

from agentic_pm.modeling.model_selection import make_windows, SequenceDataset, train_sequence_model

# ----------------------------
# 1) Load final processed dataset
# ----------------------------
df = pd.read_csv("data/processed/CMAPSS/train_FD001_final.csv")

# ----------------------------
# 2) Best hyperparameters (Fold 1 winner)
# ----------------------------
SEQ_LEN    = 96
HIDDEN     = 110
LAYERS     = 2
LR         = 0.001363
BATCH_SIZE = 32
EPOCHS     = 40
PATIENCE   = 6

print("Using final GRU hyperparameters:", {
    "seq_len": SEQ_LEN,
    "hidden": HIDDEN,
    "layers": LAYERS,
    "lr": LR,
    "batch_size": BATCH_SIZE
})

# ----------------------------
# 3) Features to use (sequence features)
# ----------------------------
assert "sequence_features" in globals(), "❌ You must define sequence_features before running this cell"

print(f"Using {len(sequence_features)} sequence features.")

# ----------------------------
# 4) Create holdout validation (Last 10% for each unit)
# ----------------------------
def per_unit_holdout_df(df, val_frac=0.10):
    train_rows = []
    val_rows = []

    for u, g in df.groupby("unit"):
        cutoff = int(len(g) * (1 - val_frac))
        train_rows.append(g.iloc[:cutoff])
        val_rows.append(g.iloc[cutoff:])

    return pd.concat(train_rows), pd.concat(val_rows)

train_df, val_df = per_unit_holdout_df(df, val_frac=0.10)

print("Train rows:", train_df.shape, "Val rows:", val_df.shape)

# ----------------------------
# 5) Create final windows
# ----------------------------
Xtr, ytr, _ = make_windows(train_df, sequence_features, seq_len=SEQ_LEN)
Xva, yva, _ = make_windows(val_df, sequence_features, seq_len=SEQ_LEN)

print("Final training windows:", Xtr.shape, "Val windows:", Xva.shape)

train_loader = DataLoader(SequenceDataset(Xtr, ytr), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(SequenceDataset(Xva, yva), batch_size=BATCH_SIZE, shuffle=False)

# ----------------------------
# 6) Define GRU model
# ----------------------------
class GRURegressor(nn.Module):
    def __init__(self, n_features, hidden_size=64, num_layers=1, dropout=0.1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        _, h_n = self.gru(x)
        last = h_n[-1]
        return self.fc(last).squeeze(1)

# instantiate model
model = GRURegressor(
    n_features=len(sequence_features),
    hidden_size=HIDDEN,
    num_layers=LAYERS,
    dropout=0.10
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Training on:", device)

# ----------------------------
# 7) Train final model
# ----------------------------
model, best_val = train_sequence_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    lr=LR,
    epochs=EPOCHS,
    patience=PATIENCE,
    device=device
)

print("Final validation MAE:", best_val)

# ----------------------------
# 8) Save model + metadata
# ----------------------------
SAVE_DIR = Path("artifacts/final_model")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

torch.save(model.state_dict(), SAVE_DIR / "gru_final_weights.pth")

metadata = {
    "model_type": "GRU",
    "seq_len": SEQ_LEN,
    "hidden": HIDDEN,
    "layers": LAYERS,
    "dropout": 0.10,
    "lr": LR,
    "batch_size": BATCH_SIZE,
    "features": sequence_features
}

joblib.dump(metadata, SAVE_DIR / "gru_final_metadata.pkl")

print("Saved final GRU model and metadata to:", SAVE_DIR)


Using final GRU hyperparameters: {'seq_len': 96, 'hidden': 110, 'layers': 2, 'lr': 0.001363, 'batch_size': 32}
Using 107 sequence features.
Train rows: (18525, 428) Val rows: (2106, 428)
Final training windows: (9025, 96, 107) Val windows: (0, 96, 107)
Training on: cuda


2025-11-23 12:12:16,970 INFO [SEQ] Epoch 00  val_MAE=nan
2025-11-23 12:12:17,792 INFO [SEQ] Epoch 01  val_MAE=nan
2025-11-23 12:12:18,656 INFO [SEQ] Epoch 02  val_MAE=nan
2025-11-23 12:12:19,658 INFO [SEQ] Epoch 03  val_MAE=nan
2025-11-23 12:12:20,638 INFO [SEQ] Epoch 04  val_MAE=nan
2025-11-23 12:12:21,627 INFO [SEQ] Epoch 05  val_MAE=nan


Final validation MAE: inf
Saved final GRU model and metadata to: artifacts\final_model
